### Inputs and Imports

In [2]:
input_file_list = []
sample_input_file_list = []

with open(r'C:\Users\andrew.morris\Documents\GitHub\advent_of_code\Advent of Code\2021\Inputs\day_05.txt', 'r') as input_file: 
    for line in input_file.readlines():
        input_file_list.append(line.rstrip('\n'))
with open(r'C:\Users\andrew.morris\Documents\GitHub\advent_of_code\Advent of Code\2021\Inputs\day_05_sample.txt', 'r') as input_file: 
    for line in input_file.readlines():
        sample_input_file_list.append(line.rstrip('\n'))        

########################        
# Part One Sample Answer:
########################
# Part Two Sample Answer:
########################

### Part One

In [5]:
# maybe approach this one by assignin each line segment to a dict instead of a list of lists
# split each segment using regex to find the first set of numbers either side of a comma 
# and the second set of numbers each side of a comma

line_segments = sample_input_file_list

for index, element in enumerate(line_segments):
    line_segments[index] = [element]
    
line_segments    

[['0,9 -> 5,9'],
 ['8,0 -> 0,8'],
 ['9,4 -> 3,4'],
 ['2,2 -> 2,1'],
 ['7,0 -> 7,4'],
 ['6,4 -> 2,0'],
 ['0,9 -> 2,9'],
 ['3,4 -> 1,4'],
 ['0,0 -> 8,8'],
 ['5,5 -> 8,2']]

['0,9 -> 5,9']
['0,9 -> 5,9', '0']


### Part Two